In [ ]:
import os
import sys
import numpy
from matplotlib import pyplot as plot
from sklearn.neighbors import NearestNeighbors

sys.path.append('./lib/caffe/python/')
import caffe

In [ ]:
caffe.set_mode_gpu()

In [ ]:
sketch_net = caffe.Net('./lib/caffe/models/triplet_googlenet/triplet_googlenet_sketch_deploy.prototxt', 
                       './lib/caffe/models/triplet_googlenet/triplet_googlenet_finegrain.caffemodel',
                       caffe.TEST)

In [ ]:
transformer = caffe.io.Transformer({'data': numpy.shape(sketch_net.blobs['data'].data)})

In [ ]:
transformer.set_mean('data', numpy.load('./lib/caffe/python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1))
transformer.set_transpose('data', (2, 0, 1))
transformer.set_channel_swap('data', (2, 1, 0))
transformer.set_raw_scale('data', 255.0)

In [ ]:
print(os.listdir('./images/sketch/'))
sketch_path = './images/sketch/' + input("Sketch name: ")
sketch = transformer.preprocess('data', caffe.io.load_image(sketch_path))
sketch_reshaped = numpy.reshape(sketch, sketch_net.blobs['data'].data.shape)

features_sketch_all_layers = sketch_net.forward(data=sketch_reshaped)
features_sketch_output_layer = numpy.copy(features_sketch_all_layers['pool5/7x7_s1_s'])

# print(features_sketch_all_layers)
# print(numpy.shape(features_sketch_output_layer))

In [ ]:
features = numpy.load('./data/features.npy')
images = numpy.load('./data/images.npy')

In [ ]:
k = 3
neighbors = NearestNeighbors(
    n_neighbors=k, 
    algorithm='brute', 
    metric='cosine'
).fit(features)
print(neighbors)

In [ ]:
distances, indices = neighbors.kneighbors(numpy.reshape(features_sketch_output_layer, [numpy.shape(features_sketch_output_layer)[1]]).reshape(1, -1))

# print(numpy.shape(features_sketch_output_layer))
# print(numpy.reshape(features_sketch_output_layer, [numpy.shape(features_sketch_output_layer)[1]]).reshape(1, -1))

In [ ]:
figure = plot.figure(0)
plot.imshow(plot.imread(sketch_path))
plot.show()

for i in range(k):
    plot.figure(i+1)
    img = plot.imread("./images/archive/" + images[indices[0][i]])
    plot.imshow(img)
    plot.show()